In [1]:
import torch

if torch.backends.mps.is_available():
    print("MPSが利用可能です！")
else:
    print("MPSは利用できません。")


MPSが利用可能です！


In [2]:
import torch

# MPSが利用可能か確認
if torch.backends.mps.is_available():
    print("MPSが利用可能です！")
else:
    print("MPSは利用できません。")


MPSが利用可能です！


In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from peft import PeftModel
from utils import *

# デバイス設定
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# モデルのロード
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    torch_dtype=torch.float32,  # float16はサポートが限定的
).to(device)

# PeftModelの設定
model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval().to(device)

# トークナイザーの設定
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

# データのロード
test_dataset = load_from_disk('./data/fingpt-forecaster-crypto-20230131-20231231-1-4-08/')['test']

# テスト関数
def test_demo(model, tokenizer, prompt):
    inputs = tokenizer(
        prompt, return_tensors='pt',
        padding=False, max_length=4096
    )
    inputs = {key: value.to(device) for key, value in inputs.items()}  # デバイスに移動

    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output

answers, gts = [], []

# 推論ループ
for i in range(len(test_dataset)):
    prompt = test_dataset[i]['prompt']
    output = test_demo(model, tokenizer, prompt)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)
    gt = test_dataset[i]['answer']
    print('\n------- Prompt ------\n')
    print(prompt)
    print('\n------- LLaMA2 Finetuned ------\n')
    print(answer)
    print('\n------- GPT4 Groundtruth ------\n')
    print(gt)
    print('\n===============\n')
    answers.append(answer)
    gts.append(gt)

calc_metrics(answers, gts)
print(answers[0])


/Users/imac27/FinGPT_2024/finGPT_env/lib/python3.10/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/Users/imac27/FinGPT_2024/finGPT_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/imac27/FinGPT_2024/finGPT_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


FileNotFoundError: Directory ./data/fingpt-forecaster-crypto-20230131-20231231-1-4-08/ not found